# Consts and imports

In [10]:
FEATURES_PATH = 'DEAM/features'
VALENCE_FILE = 'DEAM/annotations/annotations averaged per song/song_level/static_annotations_averaged_songs_1_2000.csv'
GENRE_FILE = 'DEAM/metadata/metadata_2013.csv'


import csv
import os
import pandas as pd
import numpy as np


# Calculate the average of the feature

In [11]:
def loadFeaturesFromFile (file_name):
    features_values = []
    with open('DEAM/features/' + file_name, newline='') as csvfile:
        data = list(csv.reader(csvfile, delimiter=';'))
    for i in range (1, len(data[1])):
        media = 0.0
        suma = 0.0
        for j in range (1, len(data)):
            suma += float((data[j][i]))
            media = suma / (len(data) - 1)
        features_values.append(media)
    return features_values


# Extract the name of the features

In [12]:
def extractFeaturesNames():
    with open('DEAM/features/2.csv', newline='') as csvfile:
        data = list(csv.reader(csvfile, delimiter=';'))
    features = data[0]
    features = features[1:]
    #print(len(features))
    #print(features)
    return features

features = extractFeaturesNames()

# Create the table (features, valence, arousal)

In [13]:
def createDataFile(file_data):
    
    # create the header of the matrix (features and values of 2 file)
    columns_name = ['file'] + features + ['valence'] + ['arousal']
    df = pd.DataFrame([], columns=columns_name)
    df.to_csv(file_data + ".csv")
    
    # extract the name of the files
    features_files = os.listdir(FEATURES_PATH)
    
    #extract valence values from files
    valence_matrix = pd.read_csv(VALENCE_FILE)
    valences = valence_matrix[['song_id', ' valence_mean', ' arousal_mean']]
    
    
    # fill the matrix with data
    num = 1
    song_id = 0
    for file in features_files:
        song_id = int(file[:-4])
        if song_id <= 1999:
            valence_for_song = (valences[valences['song_id'] == song_id][' valence_mean']).iloc[0] 
            arousal_for_song = (valences[valences['song_id'] == song_id][' arousal_mean']).iloc[0] 
            feat = [num] + [file] + loadFeaturesFromFile(file) + [valence_for_song] + [arousal_for_song]
            
            with open(file_data + '.csv', mode='a', newline='') as file:  
                writer = csv.writer(file)
                writer.writerow(feat)
            num = num + 1

createDataFile('recap_data')
